In [ ]:
# Import a Client
import os
import sys
import math
import matplotlib.pyplot as plt
import numpy as np
from __future__ import print_function
from IPython.display import display
from keras.layers.core import Dense
from keras.layers import LSTM
from keras.models import Sequential
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from influxdb_client import InfluxDBClient
from scipy import interpolate
import pandas as pd
sys.path.insert(0, os.path.abspath('../'))
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

# parameters to be set ("optimum" hyperparameters obtained from grid search):
look_back = 38#移动平均长度
epochs = 10#迭代多少次
batch_size = 32#默认值

# fix random seed for reproducibility
np.random.seed(7)

# read data from InfluxDB 2.0 
token = "K2TFepmH-x3a0wHiponbMFvPgg7r3BQv63ET1YqZk4glJOIHunvrw5Yjkj3XxxUaxuIsssjRfO9VA6YclLizEA=="
org = "CYTEK"
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org, debug=False)
query_api = client.query_api()
data_influxdb = query_api.query_data_frame('from(bucket:"Xiaoluzi")'
                                        '|> range(start:2023-03-28T01:44:00Z, stop:2023-04-20T07:02:00Z)'
                                        '|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")'
                                        '|> keep(columns: ["_power_A","_niv_temperature_A","_power_B","_niv_temperature_B","_time"])')

#delete niv==0
data_influxdb=data_influxdb[~data_influxdb['_niv_temperature_A'].isin([0.000])]

#interpol 1s
temp_data=data_influxdb
temp_data.index=pd.to_datetime(temp_data['_time'],unit='s')
interpol=interpolate.interp1d(temp_data['_time'], temp_data["_niv_temperature_A"], kind='linear')
temp_data['date']=temp_data.index
helper = pd.DataFrame({'date': pd.date_range(temp_data.index.min(), temp_data.index.max(),freq='1S')})
temp_data2= pd.merge(temp_data, helper, on='date', how='outer').sort_values('date')
temp_data2.index=temp_data2['date']
temp_data2["_niv_temperature_A"] = temp_data2["_niv_temperature_A"].interpolate(method='linear')
temp_data2["_niv_temperature_B"] = temp_data2["_niv_temperature_B"].interpolate(method='linear')
temp_data2["_power_A"] = temp_data2["_power_A"].interpolate(method='linear')
temp_data2["_power_B"] = temp_data2["_power_B"].interpolate(method='linear')
temp_data2=temp_data2[temp_data2['date'].isin(helper['date'])]
data_influxdb=temp_data2
print(data_influxdb)


train_data=pd.DataFrame()
train_data['P_and_T_data']=np.reshape(np.array(data_influxdb[['_niv_temperature_A',"_niv_temperature_B",'_power_A',"_power_B"]]),len(data_influxdb['_niv_temperature_A'])*4)

print(train_data['P_and_T_data'])

# save train_data values as type of floating point number
train_data_tofloat = train_data.P_and_T_data.values.astype('float32')

# reshape to column vector
train_data_tofloat = train_data_tofloat.reshape(len(train_data_tofloat), 1)

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
train_data_tofloat = scaler.fit_transform(train_data_tofloat)

# split data into training set and test set
train_size = int(len(train_data_tofloat) * 0.67)#2/3用作训练集
if train_size%2==1:
	train_size+=1
test_size = len(train_data_tofloat) - train_size
train, test = train_data_tofloat[0:train_size,:], train_data_tofloat[train_size:len(train_data_tofloat),:]

print('Split data into training set and test set... Number of training samples/ test samples:', len(train), len(test))
# convert an array of values into a time series dataset 
# in form 
#                     X                     Y
# t-look_back+1, t-look_back+2, ..., t     t+1
#print(train.head(10))
print(train[0:10])
print(train[10:12])
print(test)
def create_dataset(dataset, look_back):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		if i%4==0:
			dataX.append(dataset[i:(i+look_back), 0])
			dataY.append(dataset[i+look_back:(i+look_back+2), 0])
	return np.array(dataX), np.array(dataY)


# convert train_data into time series dataset
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

# reshape input of the LSTM to be format [samples, time steps, features]
trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
trainY = np.reshape(trainY, (trainY.shape[0], trainY.shape[1], 1))
testY = np.reshape(testY, (testY.shape[0], testY.shape[1], 1))

# create and fit the LSTM network
model = Sequential()
model.add(LSTM(4, input_shape=(look_back, 1)))#4 :Positive integer, dimensionality of the output space.
model.add(Dense(2),activation='sigmoid')#神经网络层
model.compile(loss='mse', optimizer='adam')#MeanSquaredErro #loss y(true)-y(expect)
model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size)# batch_size：Number of samples per gradient update.
model.summary()


In [2]:
model.save(r'C:\Users\Administrator\Desktop\tfmodel\TABPAB38-03-28-04-20-2023-2.h5')